In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Create Multi-layer preceptron class

In [2]:
class MLP(object):
    def __init__(self, NI, NH, NO):  
        self.in_num = NI              
        self.hidden_num = NH          
        self.out_num = NO              
        self.W1 = np.array        
        self.W2 = np.array        
        self.dW1 = np.array      
        self.dW2 = np.array      
        self.Z1 = np.array       
        self.Z2 = np.array       
        self.bias_lower = np.array       
        self.bias_upper = np.array      
        self.dBias_lower = np.array    
        self.dBias_upper = np.array    
        self.H = np.array  
        self.O = np.array  
        
        
       
    def randomise(self): 
        self.W1 = np.array((np.random.uniform(low=0, high=1, size=(self.in_num, self.hidden_num))).tolist())
        self.W2 = np.array((np.random.uniform(low=0, high=1, size=(self.hidden_num, self.out_num))).tolist())
        

        # set dW1 and dW2 to all zeroes.
        self.dW1 = np.dot(self.W1, 0)
        self.dW2 = np.dot(self.W2, 0)
        
    def softmax_function(self, x):
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps)
    
    def sigmoid_function(self, x):
        return 1 / (1 + np.exp(-x))
    def derivative_sigmoid_function(self, x):
        return np.exp(-x) / (1 + np.exp(-x)) ** 2
    
    
    def tanh_function(self, x):
        return np.tanh(x)
    def derivative_tanh_function(self, x):
        return 1 - np.tanh(x)**2
       
      
    
    def forward(self, I, activation):
   
        if activation == 'sigmoid_function':
            
            self.Z1 = np.dot(I, self.W1)
            self.H = self.sigmoid_function(self.Z1)
            self.Z2 = np.dot(self.H, self.W2)
            self.O = self.sigmoid_function(self.Z2)
    
        elif activation == 'tanh_function' :
            self.Z1 = np.dot(I, self.W1) 
            self.H = self.tanh_function(self.Z1)
            self.Z2 = np.dot(self.H, self.W2) 
            self.O = self.Z2
            
        return self.O
    
    
    
    def backward(self, I, target, activation):
        output_error = np.subtract(target, self.O)  
        if activation == 'sigmoid_function' : 
            activation_O=self.derivative_sigmoid_function(self.Z2)
            activation_H=self.derivative_sigmoid_function(self.Z1)
            
        elif activation == 'tanh_function' :
            activation_O=self.derivative_tanh_function(self.Z2)
            activation_H=self.derivative_tanh_function(self.Z1)
            
        elif activation == 'softmax_function' :
            activation_O=self.softmax_function(self.Z2)
            activation_H=self.softmax_function(self.Z1)
            
        dw2_a = np.multiply(output_error, activation_O)
        self.dW2 = np.dot(self.H.T, dw2_a)
        dw1_a=np.multiply(np.dot(dw2_a, self.W2.T), activation_H)
        self.dW1=np.dot(I.T,dw1_a)
        return np.mean(np.abs(output_error))


    
    def updateWeights(self, learningRate):
        self.W1 = np.add(self.W1,learningRate * self.dW1)
        self.W2 = np.add(self.W2,learningRate * self.dW2)
        self.dW1 = np.array
        self.dW2 = np.array

## **XOR function**

Train an MLP with 2 inputs, 3-4+ hidden units and one output on the
following examples (XOR function):

((0, 0), 0)

((0, 1), 1)

((1, 0), 1)

((1, 1), 0)


In [3]:
def XOR(inputs, outputs, max_epochs, learning_rate, NH):
    NI = 2
    NO = 1
    NN = MLP(NI, NH, NO)
    NN.randomise()
    print('\nMax Epoch: ' + str(max_epochs))
    print('\nLearning Rate: ' + str(learning_rate))
    print('\nNumber of hidden units: ' + str(NH))
    print('\nBefore Training: ')
    
    for i in range(len(inputs)):
        NN.forward(inputs[i],'sigmoid_function')
        prediction =0
        if(NN.O > 0.5):
                prediction = 1 
        print('Actual:\t {}  Output:\t {}  Prediction:\t {}'.format(str(outputs[i]), str(NN.O), str(prediction)))
    
    print('\nTraining:\n')
    
    for i in range(0, max_epochs):
        NN.forward(inputs,'sigmoid_function')
        error = NN.backward(inputs, outputs,'sigmoid_function')
        NN.updateWeights(learning_rate)

        if (i + 1) % (max_epochs / 25) == 0:
            print('Error at epoch: ' + str(i + 1) + ' is' + str(error))
            
    print('\n Test :\n')
    
    accuracy=float(0)
    for i in range(len(inputs)):
        prediction = 0
        NN.forward(inputs[i],'sigmoid_function')
        if(NN.O > 0.5):
                prediction = 1 
        print('Actual: {} Output: {}  Prediction: {}'.format(str(outputs[i]), str(NN.O), str(prediction)))
        if(outputs[i][0]==0):
            accuracy+=1-NN.O[0]
        elif(outputs[i][0]==1):
            accuracy+=NN.O[0]
            
    values_to_add = {'Learning Rate': learning_rate,'Accuracy': accuracy/4, 'Error': 1-accuracy/4}
    row_to_add = pd.Series(values_to_add, name=NH)
    
    return row_to_add

In [4]:
iteration=10000
learn_rate=[1.0,0.8,0.6,0.4,0.2,0.02]
Output_list = []
df = pd.DataFrame(columns=['Learning Rate', 'Accuracy', 'Error'])
NH=[3,6,12,24]

#### Create inputs and outputs

In [5]:
np.random.seed(1)
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
outputs = np.array([[0], [1], [1], [0]])

#### Run model

In [ ]:
for i in range(len(NH)):
    for j in range(len(learn_rate)):
        x = XOR(inputs, outputs , iteration, learn_rate[j], NH[i])
        df = df.append(x)
        print('\n\n')

In [ ]:
df[df.Error == df.Error.min()]

In [ ]:
NH3 = df[:6]
NH6 = df[6:12]
NH12 = df[12:18]
NH24 = df[18:]

# line 1 points
x1 = NH3['Learning Rate']
y1 = NH3['Error']
# plotting the line 1 points 
plt.plot(x1, y1, label = "3 Hidden units")

# line 2 points
x2 = NH6['Learning Rate']
y2 = NH6['Error']
# plotting the line 2 points 
plt.plot(x2, y2, label = "6 Hidden Units")

# line 3 points
x3 = NH12['Learning Rate']
y3 = NH12['Error']
# plotting the line 1 points 
plt.plot(x3, y3, label = "12 Hidden units")

# line 2 points
x4 = NH24['Learning Rate']
y4 = NH24['Error']
# plotting the line 2 points 
plt.plot(x4, y4, label = "24 Hidden Units")


plt.xlabel('Learning Rate')
# Set the y axis label of the current axis.
plt.ylabel('Error')
# Set a title of the current axes.
plt.title('Max epoch = 10000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

## **SIN function**

Generate 500 vectors containing 4 components each. The value of each
component should be a random number between -1 and 1. These will be
your input vectors. The corresponding output for each vector should be
the sin() of a combination of the components. Specifically, for inputs:
[x1 x2 x3 x4]
the (single component) output should be:
sin(x1-x2+x3-x4)

Train an MLP with 4 inputs, at least 5 hidden units and one output
on 400 of these examples and keep the remaining 100 for testing.

In [30]:
def SIN(inputs, outputs, max_epochs, learning_rate, NH):

    NI = 4
    NO = 1
    NN = MLP(NI, NH, NO)
    NN.randomise()
    print('\nMax Epoch: ' + str(max_epochs))
    print('\nLearning Rate: ' + str(learning_rate))
    print('\nNumber of hidden units: ' + str(NH))
    print('\nBefore Training: ')

    for i in range(400):
        NN.forward(inputs[i],'tanh_function')
        print('Target:\t{}\t Output:\t {}'.format(str(outputs[i]),str(NN.O)))
    print('Training:\n')
    
    
#    training process
    train_error=float(0)
    for i in range(0, max_epochs):
        error = 0
        NN.forward(inputs[:400],'tanh_function')
        error = NN.backward(inputs[:400], outputs[:400],'tanh_function')
        NN.updateWeights(learning_rate)
       #prints error every 5% of epochs
        if (i + 1) % (max_epochs / 25) == 0:
            print('Error at Epoch ' + str(i + 1) + ' is ' + str(error))
            train_error = error
    
    difference=float(0)
    suum = float(0)
    output = []
    expected = []
    print('\n Test :\n')
    for i in range(400, len(inputs)):
        NN.forward(inputs[i], 'tanh_function')
        print('Actual: {}  Predicted: {}'.format(str(outputs[i]), str(NN.O)))
        difference+=np.abs(outputs[i][0]-NN.O[0])
        output.append(NN.O)
        expected.append(outputs[i])

    test_error = difference/100
    print('\ntestError:{}'.format(difference/100))
    
    values_to_add = {'Learning Rate': learning_rate, 'Test Error': test_error, 'Train Error': train_error}
    row_to_add = pd.Series(values_to_add, name=NH)
    
    return row_to_add, expected, output

In [31]:
iteration=10000
learn_rate=[0.002, 0.004, 0.006, 0.008]
trainerrorlist=[]
outputlist=[]
expectedlist=[]
df1 = pd.DataFrame(columns=['Learning Rate', 'Test Error', 'Train Error'])
NH=[10,20,40,80]

#### Generate the inputs and outputs

In [32]:
np.random.seed(213)
inputs = []
outputs = []
for i in range(0, 500):
    four_inputs_vector = list(np.random.uniform(-1.0, 1.0, 4))
    four_inputs_vector = [float(four_inputs_vector[0]),float(four_inputs_vector[1]),float(four_inputs_vector[2]),float(four_inputs_vector[3])]
    inputs.append(four_inputs_vector)

inputs=np.array(inputs)

for i in range(500):
        outputs.append(np.sin([inputs[i][0] - inputs[i][1] + inputs[i][2] - inputs[i][3]]))

#### I will now run the model

In [ ]:
for i in range(len(NH)):
    for j in range(len(learn_rate)):
        x = SIN(inputs, outputs, iteration, learn_rate[j], NH[i])
        df1 = df1.append(x[0])
        expectedlist=x[1]
        outputlist= x[2]
        print('\n\n')

In [ ]:
# line 1 points
x1 =list(range(100))
y1 =  expectedlist
# plotting the line 1 points 
plt.scatter(x1, y1, label = "Actual")

# line 2 points
x2 = list(range(100))
y2 = outputlist
# plotting the line 2 points 
plt.scatter(x2, y2, label = "Predicted")



plt.xlabel('Prediction number')
# Set the y axis label of the current axis.
plt.ylabel('Value')
# Set a title of the current axes.
plt.title('sigmoid - Max epoch = 10000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
df1[df1['Test Error'] == df1['Test Error'].min()]

In [ ]:
NH5 = df1[:4]
NH10 = df1[4:8]
NH15= df1[8:12]
NH20 = df1[12:16]
NH100 = df1[16:]
# line 1 points
x1 = NH5['Learning Rate']
y1 = NH5['Test Error']
# plotting the line 1 points 
plt.plot(x1, y1, label = "10 Hidden units")

# line 2 points
x2 = NH10['Learning Rate']
y2 = NH10['Test Error']
# plotting the line 2 points 
plt.plot(x2, y2, label = "20 Hidden Units")

# line 3 points
x3 = NH15['Learning Rate']
y3 = NH15['Test Error']
# plotting the line 1 points 
plt.plot(x3, y3, label = "40 Hidden units")

# line 4 points
x4 = NH20['Learning Rate']
y4 = NH20['Test Error']
# plotting the line 1 points 
plt.plot(x4, y4, label = "80 Hidden units")

plt.xlabel('Learning Rate')
# Set the y axis label of the current axis.
plt.ylabel('Test Error')
# Set a title of the current axes.
plt.title('Max epoch = 10000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
NH5 = df1[:4]
NH10 = df1[4:8]
NH15= df1[8:12]
NH20 = df1[12:16]
NH100 = df1[16:]
# line 1 points
x1 = NH5['Learning Rate']
y1 = NH5['Train Error']
# plotting the line 1 points 
plt.plot(x1, y1, label = "10 Hidden units")

# line 2 points
x2 = NH10['Learning Rate']
y2 = NH10['Train Error']
# plotting the line 2 points 
plt.plot(x2, y2, label = "20 Hidden Units")

# line 3 points
x3 = NH15['Learning Rate']
y3 = NH15['Train Error']
# plotting the line 1 points 
plt.plot(x3, y3, label = "40 Hidden units")

# line 4 points
x4 = NH20['Learning Rate']
y4 = NH20['Train Error']
# plotting the line 1 points 
plt.plot(x4, y4, label = "80 Hidden units")

plt.xlabel('Learning Rate')
# Set the y axis label of the current axis.
plt.ylabel('Training Error')
# Set a title of the current axes.
plt.title('Max epoch = 10000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
df1[df1['Test Error'] == df1['Test Error'].min()]

In [ ]:
NH40= df1[4:8]

# line 1 points
x1 = NH40['Learning Rate']
y1 = NH40['Test Error']
# plotting the line 1 points 
plt.plot(x1, y1, label = "test error")

# line 2 points
x2 = NH40['Learning Rate']
y2 = NH40['Train Error']
# plotting the line 2 points 
plt.plot(x2, y2, label = "train error")

plt.xlabel('Learning Rate')
# Set the y axis label of the current axis.
plt.ylabel('Accuracy')
# Set a title of the current axes.
plt.title('Max epoch = 100000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

## Letter recognition

Train an MLP on the letter recognition set available in the UCI Machine
Learning repository:
http://archive.ics.uci.edu/ml/machine-learning-databases/letterrecognition/letter-recognition.data
The first entry of each line is the letter to be recognised (i.e. the
target) and the following numbers are attributes extracted from images
of the letters (i.e. your input). You can find a description of the set
here:
http://archive.ics.uci.edu/ml/datasets/Letter+Recognition
Split the dataset in a training part containing approximately 4/5 of
the records, and a testing part containing the rest.
Your MLP should have as many inputs as there are attributes (17), as
many hidden units as you want (I suggest to start at ~10) and 26
outputs (one for each letter of the alphabet).
You should train your MLP for at least 1000 epochs. After training,
check how well you can classify the data reserved for testing.



In [3]:
def letter_recognition(inputs, output, doutput, max_epochs, learning_rate, NH):
    
    #train set
    training_size = int(len(inputs) * 0.8)
    inputs_train=inputs[:training_size]
    arr = np.zeros((training_size, 26))
    for i, l in enumerate(outputs[:training_size]):
        arr[i][l] = 1
    outputs_train=arr
    
    #test set
    inputs_test=inputs[training_size:]
    
    
    #training process
    NI= 16
    NO = 26
    
    NN = MLP(NI, NH, NO)
    NN.randomise()
    print('\nMax Epoch: ' + str(max_epochs))
    print('\nLearning Rate: ' + str(learning_rate))
    print('\nHidden Units: ' + str(NH))
    print('\nTraining Process: ')
    
    df = pd.DataFrame()
    
    for i in range(0, max_epochs):
        NN.forward(inputs_train,'tanh_function')
        error = NN.backward(inputs_train, outputs_train,'tanh_function')
        NN.updateWeights(learning_rate)
    
        if (i + 1) % (max_epochs / 25) == 0:
            print(' Error at Epoch:\t' + str(i + 1) + '\t  is \t' + str(error))
    
    
    #testing process
    def to_character0(outputvector):
        listov=list(outputvector)
        a=listov.index(max(listov))
        return chr(a+ord('A'))
    
    prediction=[]
    for i in range(4000):
        NN.forward(inputs_test[i],'tanh_function')
        prediction.append(to_character0(NN.O))
        
       
    def to_character(n):
        return chr(int(n) + ord('A'))
    
    correct = {to_character(i): 0 for i in range(26)}
    
    print('==' * 30)
    # Calculate the accuracy
    accuracy = sum(correct.values()) / len(prediction)
    print('Test sample size: {} | Correctly predicted sample size: {}'.format(len(prediction),sum(correct.values()))) 
    print('Accuracy: %.3f' % accuracy)
    
    values_to_add = {'# Hidden units': NH, 'Learning rate': learning_rate, 'Accuracy': accuracy}
    row_to_add = pd.Series(values_to_add, name='x')
    df =df.append(row_to_add)
    return df

#### Generate the inputs and outputs

In [5]:
np.random.seed(1) 
inputs = []
outputs = []
doutput = []
columns=["letter","x-box","y-box","width","height","onpix","x-bar","y-bar","x2bar","y2bar","xybar","x2ybr","xy2br","x-ege","xegvy","y-ege","yegvx"]
df_accuracy = pd.DataFrame()  
df=pd.read_csv("letter-recognition.data", names=columns)
doutput=df["letter"]
length = len(df)
for i in range(length):
    outputs.append(ord(str(doutput[i]))-ord('A'))
  
 
inputs=df.drop(["letter"], axis=1)
inputs=np.array(inputs)
inputs=inputs/15

In [6]:
def change_output(output, training_size):
    arr = np.zeros((training_size, 26))
    for i, l in enumerate(output):
        arr[i][l] = 1
        
    return arr

#### I will now run model

In [ ]:
iteration=100000
learn_rate=[0.000001]
NH = [10, 15, 25]
for i in range(len(NH)):
    for j in range(len(learn_rate)):
        print('----------------------------------------------------------------------\n')
        x = letter_recognition(inputs, outputs, doutput, iteration, learn_rate[j], NH[i])
        df_accuracy = df_accuracy.append(x)
        print('\n-------------------------------------------------------------------\n')

----------------------------------------------------------------------


Max Epoch: 100000

Learning Rate: 1e-06

Hidden Units: 10

Training Process: 
 Error at Epoch:	4000	  is 	3.7134433291783058
 Error at Epoch:	8000	  is 	2.7313985341723512
 Error at Epoch:	12000	  is 	1.8483671703161313
 Error at Epoch:	16000	  is 	1.6432823892108446
 Error at Epoch:	20000	  is 	1.6002684626388062
 Error at Epoch:	24000	  is 	1.5209414394668876
 Error at Epoch:	28000	  is 	1.2870833882804817
 Error at Epoch:	32000	  is 	0.9124053272229269
 Error at Epoch:	36000	  is 	0.8366477773610058
 Error at Epoch:	40000	  is 	0.6643013496644362
 Error at Epoch:	44000	  is 	0.4554453013213272
 Error at Epoch:	48000	  is 	0.274157430094465
 Error at Epoch:	52000	  is 	0.25532245994154984
 Error at Epoch:	56000	  is 	0.07430169273162411
 Error at Epoch:	60000	  is 	0.0741193343865245
 Error at Epoch:	64000	  is 	0.07407231404032162
 Error at Epoch:	68000	  is 	0.07440650389037215
 Error at Epoch:	72000	  is 	0.0

In [ ]:
NH10 = df_accuracy[:4]
NH15 = df_accuracy[4:8]
NH25 = df_accuracy[8:12]
#NH24 = df_acc[12:]

# line 1 points
x1 = NH10['Learning rate']
y1 = NH10['Accuracy']
# plotting the line 1 points 
plt.plot(x1, y1, label = "10 Hidden units")

# line 2 points
x2 = NH15['Learning rate']
y2 = NH15['Accuracy']
# plotting the line 2 points 
plt.plot(x2, y2, label = "15 Hidden Units")

# line 3 points
x3 = NH25['Learning rate']
y3 = NH25['Accuracy']
# plotting the line 1 points 
plt.plot(x3, y3, label = "25 Hidden units")


plt.xlabel('Learning rate')
# Set the y axis label of the current axis.
plt.ylabel('Accuracy')
# Set a title of the current axes.
plt.title('Max epoch = 10000')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()